In [4]:
search_for = 123
start_from = 0
threads = 10
thread = 7
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002328634262084961
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 350465005
type: [1, 1, 1, 1, 123] 123
cases of this type: 228886641
100000 14.424822536471456
200000 24.390927648482368
300000 25.20392376956691
400000 25.074114334401767
500000 26.35136991538902
600000 25.715431720302536
700000 24.556249749598592
800000 24.73741154023651
900000 23.79121554261243
1000000 10.72529223698464
1100000 24.509726922822036
1200000 24.53321784404747
1300000 24.365198171050277
1400000 25.358452605071935
1500000 25.238489023905135
1600000 24.611886385432868
1700000 24.692456022845242
1800000 24.942785342251966
1900000 10.088334423600555
2000000 25.031018510166263
2100000 29.69349993325001
2200000 28.25278787629053
2300000 28.90076985644015
2400000 29.437961805842164
2500000 27.483951505436092
2600000 27.485906687742634
2700000 27.088129324009213
2800000 17.338454071230082
2900000 21.551112482842175
3000000

29400000 23.14339499408337
29500000 23.344543938054624
29600000 12.16281651684526
29700000 23.104580276039172
29800000 23.825175333934155
29900000 23.718775608983183
30000000 21.764784440478078
30100000 12.886314164760979
30200000 23.96126788604886
30300000 23.500939181126462
30400000 23.010503335728004
30500000 22.368705111084175
30600000 20.860239972311597
30700000 15.929665221806466
30800000 14.905078504842812
30900000 21.112476074413344
31000000 21.895246724012004
31100000 22.93384016298784
31200000 22.96937098716565
31300000 23.7646148733493
31400000 15.1825391945645
31500000 20.57599406594622
31600000 23.63437310805228
31700000 23.862227661177016
31800000 23.639881609301646
31900000 17.269659321021855
32000000 17.63751366083943
32100000 24.070832236554025
32200000 22.241319727734812
32300000 23.194834625440137
32400000 21.316050894580457
32500000 20.458134601097676
32600000 8.614459969226326
32700000 21.66031296985032
32800000 20.493445000420014
32900000 22.362914493784565
330000

59100000 11.041004780498227
59200000 19.607304228128314
59300000 19.995541173466908
59400000 20.80628267604161
59500000 20.912696781471283
59600000 22.121037032053092
59700000 21.367264696358653
59800000 20.996046342609556
59900000 20.243263928039532
60000000 21.075529348809695
60100000 9.116739755410821
60200000 18.924100024384174
60300000 18.84936342810455
60400000 19.318736647081884
60500000 9.650170298125413
60600000 17.22851109454979
60700000 19.23462773968638
60800000 18.68827683777014
60900000 16.72606255167984
61000000 13.50076517941771
61100000 19.86165777055446
61200000 20.966264459904345
61300000 20.64207877929649
61400000 20.842684650520777
61500000 21.35540230091382
61600000 20.90029067869844
61700000 20.976947275983807
61800000 19.85752856424053
61900000 16.94777216529899
62000000 11.96345506334255
62100000 18.020910876575048
62200000 19.541948689424007
62300000 18.944375799742726
62400000 7.493512663128383
62500000 18.747590470046838
62600000 18.32834144854664
62700000 1

88900000 17.144204796437464
89000000 16.879642563671627
89100000 17.37053913944468
89200000 18.42970742300239
89300000 18.572305607512558
89400000 19.060326170620773
89500000 18.970922323919623
89600000 18.087862334549097
89700000 16.999653720861744
89800000 17.141689772444813
89900000 17.689300290378643
90000000 16.657455755056386
90100000 8.331972420020492
90200000 16.56849584054129
90300000 6.415438624759568
90400000 17.504787612330933
90500000 8.853172605261877
90600000 17.252795426228385
90700000 17.327525326015554
90800000 17.381009738545945
90900000 17.12281979736814
91000000 17.667660333654823
91100000 18.6920561012455
91200000 18.842571955874284
91300000 19.269392381441275
91400000 18.299501909878103
91500000 16.898080916692095
91600000 16.753307537477504
91700000 17.20167824327446
91800000 17.001835841184597
91900000 16.114984164423852
92000000 8.65393280340291
92100000 13.559377785051467
92200000 10.27053146114543
92300000 11.433521671540584
92400000 13.710967951491353
92500

118000000 1.7189522977678477
118100000 1.7290955929788385
118200000 1.7198342824631367
118300000 1.729249918774287
118400000 1.7162837609587014
118500000 1.717284600987918
118600000 1.7093116927457057
118700000 1.707168659214202
118800000 1.7152802763842145
118900000 1.711438627665758
119000000 1.7069949157121747
119100000 2.543887941791243
119200000 22.71262520366482
119300000 21.424867296650493
119400000 21.281243596601374
119500000 20.905034060899027
119600000 20.780202202246688
119700000 20.37861351554936
119800000 19.830111551411658
119900000 20.528387837756217
120000000 10.58974658934735
120100000 2.644012255117069
120200000 19.304848887282706
120300000 19.700746583049938
120400000 20.37168909978635
120500000 19.92919659480796
120600000 21.469558665747034
120700000 20.788297181727827
120800000 21.499040276057254
120900000 21.620600434121638
121000000 21.24704826006635
121100000 17.64496698414241
121200000 17.33281918198106
121300000 16.332603020976592
121400000 16.943134529983535

...

233000000 1.980638166911198
233100000 2.151131310009943
233200000 2.4774487659619453
233300000 2.050428309522205
233400000 2.048923421746499
233500000 1.7543953967002135
233600000 0.9045419754852752
233700000 2.8339422016348808
233800000 2.559837763415366
233900000 1.9871690131664739
234000000 2.0364206198747126
234100000 1.9814261546132863
234200000 2.0510779172898266
234300000 2.056331916798658
234400000 1.8864197268068992
234500000 2.277440185215546
234600000 2.3489023177320028
234700000 2.653522352236837
234800000 2.42124300168001
234900000 2.183731670833975
235000000 2.2230330921407044
235100000 2.160288852194412
235200000 1.9759298691825802
235300000 2.9729299336758856
235400000 0.8812271859909826
235500000 0.7199016947686109
235600000 2.954612240660975
235700000 1.8998078815822634
235800000 2.0041122757854097
235900000 2.0230964307291806
236000000 2.0428409750225045
236100000 2.086225237641792
236200000 2.737833064367255
236300000 2.275836195691662
236400000 2.411845085849676
23

261600000 1.761030444375485
261700000 3.0475736123428545
261800000 3.2818530381698743
261900000 3.117324220380601
262000000 3.1634174172819853
262100000 3.0837129876361256
262200000 3.061423862661845
262300000 3.107924168661445
262400000 3.766246691437244
262500000 3.831773642118464
262600000 3.3551894371295075
262700000 3.0648425885570014
262800000 3.1186349841344096
262900000 3.261616623900056
263000000 3.205474613595714
263100000 2.9957638249744107
263200000 3.374288836450835
263300000 0.7274668343425625
263400000 0.741707914704346
263500000 2.7799625833588344
263600000 3.2491448507539165
263700000 3.1825024894286726
263800000 3.3775650768436787
263900000 3.2158067103994226
264000000 2.9170855491540104
264100000 3.243486310730378
264200000 3.206521500414338
264300000 4.404213032590405
264400000 3.9919605438282586
264500000 3.544737481187605
264600000 3.335350146236188
264700000 3.5735318214144405
264800000 3.741722460221546
264900000 3.4437778171787494
265000000 3.099718196733604
26

290400000 2.4225839950851
290500000 2.1009709049050014
290600000 2.091672711400082
290700000 1.925335881545077
290800000 2.172393265161306
290900000 2.052012283834166
291000000 2.0340808908669885
291100000 2.07575219950668
291200000 0.7947172631546789
291300000 0.50141003607538
291400000 1.8087999595122835
291500000 1.8810690196307929
291600000 2.073660769053913
291700000 2.130320002029151
291800000 2.1150470101937486
291900000 1.9107279122745031
292000000 2.1082506433057886
292100000 2.2374447782221405
292200000 2.514993792280803
292300000 2.0995658935117123
292400000 2.143883604720394
292500000 1.8826730041814084
292600000 2.0085172763233183
292700000 2.059725093044251
292800000 2.1395425773017904
292900000 1.9343238465751706
293000000 1.6984405725207394
293100000 0.48753775814462036
293200000 0.49379287575419745
293300000 2.356289453280439
293400000 1.9147932474836542
293500000 2.0334806451687815
293600000 1.9965658828691277
293700000 2.1258038159058925
293800000 1.8869279909692607


318100000 1.1745105948062242
318200000 1.127647759126524
318300000 1.1571788910642002
318400000 1.1971868352619808
318500000 1.0676951352236437
318600000 1.2742965944164033
318700000 1.1949623589765133
318800000 1.131884504113078
318900000 1.123005027311633
319000000 1.1259583099773527
319100000 1.046129253410409
319200000 1.2552488904313477
319300000 1.1319469410181344
319400000 1.0600482881262467
319500000 1.1334670508946312
319600000 1.069658292074472
319700000 1.088976385520786
319800000 1.237220626665251
319900000 1.0572793493798076
320000000 1.149939593099634
320100000 1.1261963328551947
320200000 1.0511701477331021
320300000 1.112554040108694
320400000 1.160698740606732
320500000 1.1121261862765253
320600000 1.0888587073245877
320700000 1.0707608264374866
320800000 1.02582880736957
320900000 0.9977242370776269
321000000 1.0960480515081021
321100000 1.2120528113172848
321200000 1.0612498513657715
321300000 1.1041734373336998
321400000 1.0243501921322047
321500000 1.09407118999319

345100000 0.23946877382229767
345200000 0.1659896975859768
345300000 0.2364752793402374
345400000 0.166410223193417
345500000 0.2254406642968688
345600000 0.17019283316093684
345700000 0.2034744483053982
345800000 0.17903340553505223
345900000 0.17141876006965834
346000000 0.17526089842025439
346100000 0.14519091738817427
346200000 0.18618284804535576
346300000 0.1412417037775119
346400000 0.1859700943057537
346500000 0.14714662226450775
346600000 0.16070478152522447
346700000 0.16662802484933534
346800000 0.13130294089397787
346900000 0.1512098700589041
347000000 0.13283587112376755
347100000 0.1555990945552091
347200000 0.1464617327232361
347300000 0.14290407107661499
best so far: 0
type: [1, 3, 1, 41, 1] 123
cases of this type: 206763
347400000 0.08414475760930445
347500000 0.0202088253249228
best so far: 0
type: [1, 3, 41, 1, 1] 123
cases of this type: 5043
best so far: 0
type: [1, 41, 1, 1, 3] 123
cases of this type: 3321
best so far: 0
type: [1, 41, 1, 3, 1] 123
cases of this typ